In [8]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam

# Set random seed
np.random.seed(0)

In [9]:
data = pd.read_csv('CF_data.csv')
data

,Company,Time,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Price
0,YUM,2015-12-01,1283.0,1260.0,-199.0,-1089.0,-28.0,402.4,498.0,1.88,52.53
1,VAR,2010-10-01,360.4,460.8,-74.8,-422.2,-33.3,360.7,363.3,3.23,55.98
2,HRL,2015-10-01,686.1,992.0,-900.9,-70.6,13.1,826.2,834.4,1.61,33.77
3,EPD,2010-12-01,320.8,2300.0,-3251.6,961.1,10.2,-338.8,124.9,0.47,20.80
4,EONGY,2010-12-01,7848.5,14864.2,1013.7,-13307.4,2592.0,-1843.9,606.7,2.24,30.41
5,CF,2012-12-01,1848.7,2375.6,-513.5,-796.8,1067.9,1616.9,1701.4,5.80,40.63
6,BDX,2014-09-01,1185.0,1746.0,-948.0,-807.0,-29.0,949.6,1034.0,5.65,113.81
7,BIIB,2013-12-01,1862.3,2345.1,-1604.7,-716.5,31.9,-1067.7,-1047.8,-4.91,257.51
8,MTRN,2010-12-01,46.4,31.0,-46.5,19.8,3.9,-27.4,-26.3,-1.12,38.64
9,NOC,2010-12-01,2053.0,2453.0,-760.0,-1266.0,427.0,-637.3,-469.1,6.29,58.77


In [10]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)
data

Net Income
Done
Cash from Operations
Done
Cash from Investing
Done
Cash from Financing
Done
Net Change in Cash
Done
Levered Free Cash Flow
Done
Unlevered Free Cash Flow
Done
Free Cash Flow / Share
Done


,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Price
0,0.195162,0.101904,0.584679,0.668242,0.474163,0.459175,0.452369,0.003819,52.53
1,0.187545,0.092933,0.585543,0.673199,0.474108,0.458964,0.451685,0.003884,55.98
2,0.190234,0.098895,0.579795,0.675813,0.474591,0.461320,0.454078,0.003806,33.77
3,0.187218,0.113578,0.563438,0.683483,0.474561,0.455424,0.450474,0.003751,20.80
4,0.249369,0.254616,0.593118,0.577408,0.501464,0.447806,0.452921,0.003837,30.41
5,0.199833,0.114427,0.582491,0.670415,0.485583,0.465322,0.458481,0.004008,40.63
6,0.194353,0.107359,0.579467,0.670339,0.474153,0.461945,0.455091,0.004001,113.81
7,0.199945,0.114084,0.574898,0.671011,0.474787,0.451734,0.444518,0.003492,257.51
8,0.184952,0.088108,0.585740,0.676485,0.474495,0.457000,0.449706,0.003675,38.64
9,0.201519,0.115296,0.580776,0.666926,0.478904,0.453913,0.447457,0.004032,58.77


In [11]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:8] # Parameters
y = NpMatrix[:,8] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

X: [[0.19516198 0.10190382 0.58467919 ... 0.45917516 0.45236896 0.00381927]
 [0.18754474 0.09293251 0.58554341 ... 0.4589641  0.45168483 0.00388429]
 [0.19023381 0.09889542 0.57979515 ... 0.46132016 0.4540775  0.00380627]
 ...
 [0.19813425 0.10176912 0.58603606 ... 0.45923792 0.45299671 0.00382891]
 [0.20573002 0.12755377 0.58208374 ... 0.45472016 0.44804477 0.00381687]
 [0.19787004 0.0888824  0.57738688 ... 0.46291752 0.45705425 0.00369453]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 8
Total Number of Training instances: 5000


[52.53 55.98 33.77 ... 63.33 53.15 40.29]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 52.53
Total number of y values 5000


In [16]:
number_of_features = len(X[1])

#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=64, activation='relu', input_shape=(number_of_features,)))
    
        #Add Dropout Layer
    network.add(layers.Dropout(0.3))

    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [17]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 1000, 
                                 batch_size=5)

In [18]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/1000
4000/4000 [==============================] - 3s 703us/sample - loss: 56.6993 - mae: 56.6993 - mse: 22269.0527
Epoch 2/1000
4000/4000 [==============================] - 2s 448us/sample - loss: 48.2331 - mae: 48.2331 - mse: 20534.8711
Epoch 3/1000
4000/4000 [==============================] - 2s 378us/sample - loss: 48.2765 - mae: 48.2765 - mse: 20475.0293
Epoch 4/1000
4000/4000 [==============================] - 2s 376us/sample - loss: 48.2111 - mae: 48.2111 - mse: 20505.8730
Epoch 5/1000
4000/4000 [==============================] - 2s 575us/sample - loss: 48.1211 - mae: 48.1211 - mse: 20368.4727
Epoch 6/1000
4000/4000 [==============================] - 2s 440us/sample - loss: 48.2961 - mae: 48.2961 - mse: 20511.1465
Epoch 7/1000
4000/4000 [==============================] - 2s 388us/sample - loss: 48.2471 - mae: 48.2471 - mse: 20491.8770
Epoch 8/1000
4000/4000 [==============================] - 2s 504us/sample - loss: 48.0296 - mae: 48.0296 - mse: 20427

4000/4000 [==============================] - 1s 305us/sample - loss: 47.9331 - mae: 47.9331 - mse: 20397.0195
Epoch 67/1000
4000/4000 [==============================] - 2s 379us/sample - loss: 48.0299 - mae: 48.0299 - mse: 20402.0586
Epoch 68/1000
4000/4000 [==============================] - 1s 330us/sample - loss: 47.6764 - mae: 47.6764 - mse: 20345.2930
Epoch 69/1000
4000/4000 [==============================] - 1s 368us/sample - loss: 47.8344 - mae: 47.8344 - mse: 20344.6934
Epoch 70/1000
4000/4000 [==============================] - 1s 309us/sample - loss: 47.8133 - mae: 47.8133 - mse: 20321.7129
Epoch 71/1000
4000/4000 [==============================] - 1s 340us/sample - loss: 47.7007 - mae: 47.7007 - mse: 20293.8828
Epoch 72/1000
4000/4000 [==============================] - 2s 375us/sample - loss: 47.9244 - mae: 47.9244 - mse: 20394.5293
Epoch 73/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 47.8598 - mae: 47.8597 - mse: 20376.7109
Epoch 74/1000
4000/400

4000/4000 [==============================] - 1s 367us/sample - loss: 47.8625 - mae: 47.8625 - mse: 20365.1094
Epoch 132/1000
4000/4000 [==============================] - 1s 319us/sample - loss: 47.7343 - mae: 47.7344 - mse: 20363.8535
Epoch 133/1000
4000/4000 [==============================] - 1s 318us/sample - loss: 47.7603 - mae: 47.7603 - mse: 20371.0293
Epoch 134/1000
4000/4000 [==============================] - 1s 352us/sample - loss: 47.6943 - mae: 47.6943 - mse: 20347.3223
Epoch 135/1000
4000/4000 [==============================] - 2s 406us/sample - loss: 47.5118 - mae: 47.5118 - mse: 20279.0469
Epoch 136/1000
4000/4000 [==============================] - 1s 314us/sample - loss: 47.8524 - mae: 47.8524 - mse: 20375.2129
Epoch 137/1000
4000/4000 [==============================] - 1s 303us/sample - loss: 47.6319 - mae: 47.6319 - mse: 20342.3242
Epoch 138/1000
4000/4000 [==============================] - 1s 304us/sample - loss: 47.6475 - mae: 47.6475 - mse: 20312.1211
Epoch 139/1000


4000/4000 [==============================] - 1s 327us/sample - loss: 47.4036 - mae: 47.4036 - mse: 20339.2617
Epoch 197/1000
4000/4000 [==============================] - 1s 321us/sample - loss: 47.5542 - mae: 47.5542 - mse: 20324.0332
Epoch 198/1000
4000/4000 [==============================] - 1s 311us/sample - loss: 47.4419 - mae: 47.4419 - mse: 20306.3848
Epoch 199/1000
4000/4000 [==============================] - 1s 373us/sample - loss: 47.6062 - mae: 47.6063 - mse: 20371.8613
Epoch 200/1000
4000/4000 [==============================] - 2s 403us/sample - loss: 47.3112 - mae: 47.3112 - mse: 20339.8340
Epoch 201/1000
4000/4000 [==============================] - 1s 323us/sample - loss: 47.1830 - mae: 47.1830 - mse: 20237.6523
Epoch 202/1000
4000/4000 [==============================] - 1s 327us/sample - loss: 47.4632 - mae: 47.4632 - mse: 20326.7383
Epoch 203/1000
4000/4000 [==============================] - 1s 357us/sample - loss: 47.4713 - mae: 47.4713 - mse: 20304.4297
Epoch 204/1000


4000/4000 [==============================] - 1s 306us/sample - loss: 47.3224 - mae: 47.3224 - mse: 20363.7070
Epoch 262/1000
4000/4000 [==============================] - 1s 313us/sample - loss: 47.2742 - mae: 47.2742 - mse: 20306.2207
Epoch 263/1000
4000/4000 [==============================] - 1s 321us/sample - loss: 47.3732 - mae: 47.3732 - mse: 20325.5195
Epoch 264/1000
4000/4000 [==============================] - 1s 344us/sample - loss: 47.2474 - mae: 47.2474 - mse: 20320.8730
Epoch 265/1000
4000/4000 [==============================] - 1s 344us/sample - loss: 47.2731 - mae: 47.2731 - mse: 20320.7949
Epoch 266/1000
4000/4000 [==============================] - 2s 392us/sample - loss: 47.1728 - mae: 47.1728 - mse: 20302.5059
Epoch 267/1000
4000/4000 [==============================] - 1s 363us/sample - loss: 47.2433 - mae: 47.2433 - mse: 20311.8828
Epoch 268/1000
4000/4000 [==============================] - 1s 364us/sample - loss: 47.0707 - mae: 47.0707 - mse: 20246.4355
Epoch 269/1000


...

4000/4000 [==============================] - 2s 387us/sample - loss: 46.7641 - mae: 46.7641 - mse: 20780.3359
Epoch 718/1000
4000/4000 [==============================] - 2s 395us/sample - loss: 46.7491 - mae: 46.7491 - mse: 20785.7324
Epoch 719/1000
4000/4000 [==============================] - 1s 364us/sample - loss: 46.9030 - mae: 46.9030 - mse: 20784.2227
Epoch 720/1000
4000/4000 [==============================] - 1s 361us/sample - loss: 46.8808 - mae: 46.8808 - mse: 20778.4766
Epoch 721/1000
4000/4000 [==============================] - 1s 356us/sample - loss: 46.9193 - mae: 46.9193 - mse: 20777.0879
Epoch 722/1000
4000/4000 [==============================] - 2s 387us/sample - loss: 46.8374 - mae: 46.8373 - mse: 20789.2715
Epoch 723/1000
4000/4000 [==============================] - 1s 347us/sample - loss: 46.6545 - mae: 46.6545 - mse: 20749.7031
Epoch 724/1000
4000/4000 [==============================] - 1s 374us/sample - loss: 46.8684 - mae: 46.8684 - mse: 20765.2227
Epoch 725/1000


4000/4000 [==============================] - 1s 350us/sample - loss: 46.7589 - mae: 46.7589 - mse: 20750.1914
Epoch 783/1000
4000/4000 [==============================] - 1s 369us/sample - loss: 46.7449 - mae: 46.7449 - mse: 20724.4824
Epoch 784/1000
4000/4000 [==============================] - 1s 352us/sample - loss: 46.6810 - mae: 46.6810 - mse: 20729.5781
Epoch 785/1000
4000/4000 [==============================] - 2s 396us/sample - loss: 46.6649 - mae: 46.6649 - mse: 20737.7637
Epoch 786/1000
4000/4000 [==============================] - 1s 345us/sample - loss: 46.8453 - mae: 46.8453 - mse: 20773.7676
Epoch 787/1000
4000/4000 [==============================] - 2s 375us/sample - loss: 46.6796 - mae: 46.6796 - mse: 20755.1426
Epoch 788/1000
4000/4000 [==============================] - 1s 369us/sample - loss: 46.7020 - mae: 46.7020 - mse: 20806.8359
Epoch 789/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 46.6578 - mae: 46.6578 - mse: 20717.5195
Epoch 790/1000


4000/4000 [==============================] - 1s 296us/sample - loss: 46.6109 - mae: 46.6109 - mse: 20735.7754
Epoch 847/1000
4000/4000 [==============================] - 1s 257us/sample - loss: 46.8183 - mae: 46.8182 - mse: 20801.0781
Epoch 848/1000
4000/4000 [==============================] - 1s 262us/sample - loss: 46.6561 - mae: 46.6561 - mse: 20706.4512
Epoch 849/1000
4000/4000 [==============================] - 1s 271us/sample - loss: 46.5924 - mae: 46.5924 - mse: 20688.6016- loss: 39.0057 - mae: 39
Epoch 850/1000
4000/4000 [==============================] - 1s 259us/sample - loss: 46.5933 - mae: 46.5933 - mse: 20706.5508
Epoch 851/1000
4000/4000 [==============================] - 1s 304us/sample - loss: 46.6698 - mae: 46.6698 - mse: 20756.9844
Epoch 852/1000
4000/4000 [==============================] - 1s 260us/sample - loss: 46.5989 - mae: 46.5989 - mse: 20768.2812
Epoch 853/1000
4000/4000 [==============================] - 1s 271us/sample - loss: 46.8125 - mae: 46.8125 - mse: 2

4000/4000 [==============================] - 1s 259us/sample - loss: 46.6532 - mae: 46.6533 - mse: 20745.5312
Epoch 912/1000
4000/4000 [==============================] - 1s 272us/sample - loss: 46.6812 - mae: 46.6813 - mse: 20703.2891
Epoch 913/1000
4000/4000 [==============================] - 1s 260us/sample - loss: 46.4678 - mae: 46.4678 - mse: 20720.0332
Epoch 914/1000
4000/4000 [==============================] - 1s 276us/sample - loss: 46.7684 - mae: 46.7684 - mse: 20818.9551
Epoch 915/1000
4000/4000 [==============================] - 1s 301us/sample - loss: 46.6180 - mae: 46.6180 - mse: 20726.2734
Epoch 916/1000
4000/4000 [==============================] - 1s 270us/sample - loss: 46.4983 - mae: 46.4983 - mse: 20682.4141
Epoch 917/1000
4000/4000 [==============================] - 1s 300us/sample - loss: 46.7155 - mae: 46.7156 - mse: 20703.6230
Epoch 918/1000
4000/4000 [==============================] - 1s 266us/sample - loss: 46.7585 - mae: 46.7585 - mse: 20783.6270
Epoch 919/1000


4000/4000 [==============================] - 1s 239us/sample - loss: 46.6967 - mae: 46.6967 - mse: 20719.4219
Epoch 976/1000
4000/4000 [==============================] - 1s 220us/sample - loss: 46.7396 - mae: 46.7396 - mse: 20753.6406
Epoch 977/1000
4000/4000 [==============================] - 1s 219us/sample - loss: 46.6646 - mae: 46.6647 - mse: 20726.7188
Epoch 978/1000
4000/4000 [==============================] - 1s 222us/sample - loss: 46.7901 - mae: 46.7901 - mse: 20774.1641
Epoch 979/1000
4000/4000 [==============================] - 1s 209us/sample - loss: 46.5174 - mae: 46.5174 - mse: 20720.5938
Epoch 980/1000
4000/4000 [==============================] - 1s 211us/sample - loss: 46.5237 - mae: 46.5237 - mse: 20718.4688
Epoch 981/1000
4000/4000 [==============================] - 1s 234us/sample - loss: 46.6183 - mae: 46.6183 - mse: 20727.2363
Epoch 982/1000
4000/4000 [==============================] - 1s 213us/sample - loss: 46.6162 - mae: 46.6162 - mse: 20719.1328
Epoch 983/1000


In [19]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-45.517336   -43.68352834 -51.55442595 -46.32797834 -42.98746532]
Mean MAE: -46.014146790504455
Standard Deviation of the mean: 3.021012660734181
